In [73]:
from advent.intcode import run

items = ['ornament', 'loom', 'spool of cat6', 'wreath', 'fixed point', 'shell', 'candy cane', 'weather machine']

initial = 'east->take loom->east->take fixed point->north->take spool of cat6->north->take weather machine->south->west->take shell->east->south->west->south->take ornament->west->north->take candy cane->south->east->north->west->north->take wreath->north->east'
initial = initial.replace('->', '\n')

def take_or_drop(i, action):
    #print(f"Now {action}ing {i}")
    assert 0 <= i <= 255
    assert action in ['take', 'drop']
    # i goes from 0 to 256
    b_list = [int(x) for x in list('{0:0b}'.format(i))]
    b_list = [0] * (8 - len(b_list)) + b_list
    return '\n'.join([f'{action} {items[i]}' for i in range(8) if b_list[i]])

# Basically, are going to loop from 0 to 255 and drop all our items from the previous step,
# then take a certain amount of items depending on the state, then go south
# if it's too heavy or too light, we increment the state and try again

# Note: in hindsight it turned out the whole 'handle_output' function wasn't even needed
# you can literally just hardcode the ENTIRE sequence of 256 drop/pickup actions
# because the intcode program will halt when you have the correct items
# so if you literally deleted the self.handle_output function (and fixed some syntax errors)
# it would work as well, I just decided to keep it in since it's reflective of how I solved the problem

In [74]:
class IO():
    def __init__(self, input_=[]):
        self.input_buffer = input_.copy()
        self.output_buffer = []
        self.current_state = 0 # which items we want

    def add_input(self, value):
        for c in value + '\n':
            self.input_buffer.append(ord(c))
        return self

    def read(self):
        if len(self.input_buffer) == 0:
            result = self.handle_output()
            if result == 0 and self.current_state > 0:
                raise RuntimeError(f"We did it! State={self.current_state}")
            # increment current state and try again
            self.add_input(take_or_drop(self.current_state, 'drop'))
            self.current_state += 1
            self.add_input(take_or_drop(self.current_state, 'take'))
            self.add_input('south')
        return self.input_buffer.pop(0)
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer
    
    def handle_output(self):
        out = ''.join([chr(v) for v in self.output()[:-1]])
        self.last_output = out
        self.output_buffer = []
        too_heavy = 'Alert! Droids on this ship are lighter than the detected' in out
        too_light = 'Alert! Droids on this ship are heavier than the detected' in out
        return 1 if too_heavy else (-1 if too_light else 0)

In [75]:
import advent
data = advent.get_intcode(25)
io = IO()
io.add_input(initial) # pick up all items and go to the security terminal
io.add_input(take_or_drop(255, 'drop')) # start with empty hands
_ = run(data.copy(), io)
io.handle_output()

# 307 is hardcoded, just didn't want to dump the entire output (which includes stuff like 'you dropped the shell', etc, etc)
print(io.last_output[-307:])

A loud, robotic voice says "Analysis complete! You may proceed." and you enter the cockpit.
Santa notices your small droid, looks puzzled for a moment, realizes what has happened, and radios your ship directly.
"Oh, hello! You should be able to get in by typing 352325632 on the keypad at the main airlock."
